In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import csv

In [2]:
train_dataset = 'data/train_dataset.csv'
test_dataset = 'data/test_dataset.csv' ## 저번시간에 사용한 csv 다시 활용

In [3]:
import pandas as pd
reading = pd.read_csv('data/train_dataset.csv')
reading.tail()
### classes 가 0과 1로 이루어진 binary classification 
## softmax는 다항 multiply classification 

,1,0,1.1,2,3,4,5,6,7,8,9
894,0,9,8,7,6,5,4,3,2,1,0
895,1,0,1,2,3,4,5,6,7,8,9
896,0,9,8,7,6,5,4,3,2,1,0
897,1,0,1,2,3,4,5,6,7,8,9
898,0,9,8,7,6,5,4,3,2,1,0


In [4]:
train_dataset = tf.contrib.data.TextLineDataset(train_dataset).batch(20).repeat(10000)
test_dataset = tf.contrib.data.TextLineDataset(test_dataset).batch(20).repeat(99999)

train_itr = train_dataset.make_one_shot_iterator()
test_itr = test_dataset.make_one_shot_iterator()

train_batch = train_itr.get_next()
test_batch = test_itr.get_next()

train_batch = tf.decode_csv(train_batch, [[0]]*11)
test_batch = tf.decode_csv(test_batch, [[0]]*11)


Instructions for updating:
Use `tf.data.TextLineDataset`.


In [5]:
train_label = tf.reshape(train_batch[0],[-1,1])
train_label = tf.cast(train_label, tf.float32)

test_label = tf.reshape(test_batch[0],[-1,1])
test_label = tf.cast(test_label, tf.float32)


In [6]:
train_feature = tf.stack(train_batch[1:],axis = 1)
train_feature = tf.cast(train_feature, tf.float32)

test_feature = tf.stack(test_batch[1:], axis = 1)
test_feature = tf.cast(test_feature, tf.float32)

print('train_label:',train_label)
print('train_feature: ',train_feature)
print('*'*70)
print('test_label:',test_label)
print('test_feature:',test_feature)


train_label: Tensor("Cast:0", shape=(?, 1), dtype=float32)
train_feature:  Tensor("Cast_2:0", shape=(?, 10), dtype=float32)
**********************************************************************
test_label: Tensor("Cast_1:0", shape=(?, 1), dtype=float32)
test_feature: Tensor("Cast_3:0", shape=(?, 10), dtype=float32)


In [7]:
def layers(x, activation = None, reuse = False):
    
    
    layer1 = tf.layers.dense(x, units = 5, activation = activation, reuse = reuse, name = 'layer1')
    layer2 = tf.layers.dense(layer1, units = 6, activation = activation, reuse =reuse, name = 'layer2')
    layer3 = tf.layers.dense(layer2, units = 7, activation = activation, reuse =reuse, name = 'layer3')
    layer4 = tf.layers.dense(layer3, units = 8, activation = activation, reuse =reuse, name = 'layer4')
    out = tf.layers.dense(layer4, units = 1, reuse =reuse, name = 'outlayer')
    
    return out 

train_out = layers(train_feature)
test_out = layers(test_feature, reuse = True)

print(train_out)
print(test_out)


Tensor("outlayer/BiasAdd:0", shape=(?, 1), dtype=float32)
Tensor("outlayer_2/BiasAdd:0", shape=(?, 1), dtype=float32)


In [8]:
for v in tf.trainable_variables():
    print(v)

<tf.Variable 'layer1/kernel:0' shape=(10, 5) dtype=float32_ref>
<tf.Variable 'layer1/bias:0' shape=(5,) dtype=float32_ref>
<tf.Variable 'layer2/kernel:0' shape=(5, 6) dtype=float32_ref>
<tf.Variable 'layer2/bias:0' shape=(6,) dtype=float32_ref>
<tf.Variable 'layer3/kernel:0' shape=(6, 7) dtype=float32_ref>
<tf.Variable 'layer3/bias:0' shape=(7,) dtype=float32_ref>
<tf.Variable 'layer4/kernel:0' shape=(7, 8) dtype=float32_ref>
<tf.Variable 'layer4/bias:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'outlayer/kernel:0' shape=(8, 1) dtype=float32_ref>
<tf.Variable 'outlayer/bias:0' shape=(1,) dtype=float32_ref>


In [9]:
saver = tf.train.Saver() 

In [10]:
loss = tf.losses.softmax_cross_entropy(train_label, train_out) ## 만약 softmax를 activation fuction으로 쓰고싶으면 cost 함수도 softmax를 써야함 
train_op = tf.train.GradientDescentOptimizer(1e-2).minimize(loss)

In [11]:
tf.summary.scalar ('loss', loss) ## loss 에 대해서 summary collecting
merged = tf.summary.merge_all()

In [12]:
pred1 = tf.nn.sigmoid(test_out) 
pred2 = tf.nn.softmax(test_out)## prediction 
accuracy1 = tf.metrics.accuracy(test_label, tf.round(pred1))
accuracy2 = tf.metrics.accuracy(test_label, tf.round(pred2))

In [13]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) 
    sess.run(tf.local_variables_initializer())
    
    writer = tf.summary.FileWriter('data/',sess.graph)
    
    for i in range(1000):
        _,_loss, = sess.run([train_op,loss])
        _summ = sess.run(merged)
    
        _pred1 = sess.run(pred1)
        _pred2 = sess.run(pred2)
        _accuracy1 = sess.run(accuracy1)
        _accuracy2 = sess.run(accuracy2)
        writer.add_summary(_summ,i)
        
        if i %100 ==0:
            print('step:{} ,loss:{}, Sigmoid accuracy:{}, Softmax accuracy:{}'.format(i, _loss, _accuracy1,_accuracy2))
            
    saver.save(sess,'data/saved parameters') 

step:0 ,loss:1.2892146110534668, Sigmoid accuracy:(0.0, 0.5), Softmax accuracy:(0.0, 0.5)
step:100 ,loss:0.0011753430590033531, Sigmoid accuracy:(0.995, 0.99504948), Softmax accuracy:(0.5, 0.5)
step:200 ,loss:0.0005422182730399072, Sigmoid accuracy:(0.9975, 0.99751246), Softmax accuracy:(0.5, 0.5)
step:300 ,loss:0.0003448121715337038, Sigmoid accuracy:(0.99833333, 0.99833888), Softmax accuracy:(0.5, 0.5)
step:400 ,loss:0.00025014509446918964, Sigmoid accuracy:(0.99874997, 0.99875313), Softmax accuracy:(0.5, 0.5)
step:500 ,loss:0.00019505596719682217, Sigmoid accuracy:(0.99900001, 0.99900198), Softmax accuracy:(0.5, 0.5)
step:600 ,loss:0.00015920696023385972, Sigmoid accuracy:(0.99916667, 0.99916804), Softmax accuracy:(0.5, 0.5)
step:700 ,loss:0.0001341066526947543, Sigmoid accuracy:(0.9992857, 0.99928671), Softmax accuracy:(0.5, 0.5)
step:800 ,loss:0.00011559783160919324, Sigmoid accuracy:(0.99937499, 0.99937576), Softmax accuracy:(0.5, 0.5)
step:900 ,loss:0.00010141347593162209, Sigmo